## Importing the elementTree library and initializing the global variables

In [ ]:
import xml.etree.ElementTree as ET
from lxml import etree

In [2]:
def initialize(et_elements):
  AUTOSAR = ET.Element("AUTOSAR")
  AUTOSAR.set("xmlns", "http://autosar.org/schema/r4.0")
  AUTOSAR.set("xmlns:xsi", "http://www.w3.org/2001/XMLSchema-instance")
  AUTOSAR.set("xsi:schemaLocation", "http://autosar.org/schema/r4.0 AUTOSAR_00046.xsd")

  AR_PACKAGES = ET.Element("AR-PACKAGES")
  AR_PACKAGE = ET.Element("AR-PACKAGE")

  AUTOSAR.append(AR_PACKAGES)
  AR_PACKAGES.append(AR_PACKAGE)

  AUTOSAR_CONF = ET.SubElement(AR_PACKAGE, "SHORT-NAME")
  AUTOSAR_CONF.text = "AutosarConfigurator"

  ELEMENTS = ET.Element("ELEMENTS")
  AR_PACKAGE.append(ELEMENTS)

  ECUC_MODULE_CONFIGURATION_VALUES = ET.Element("ECUC-MODULE-CONFIGURATION-VALUES")
  ECUC_MODULE_CONFIGURATION_VALUES.set("UUID", "88315b75-2e55-4e07-8a67-e958b2d4694f")
  ELEMENTS.append(ECUC_MODULE_CONFIGURATION_VALUES)

  CanNm = ET.SubElement(ECUC_MODULE_CONFIGURATION_VALUES, "SHORT-NAME")
  CanNm.text = "CanNm"

  DEFINITION_REF = ET.SubElement(ECUC_MODULE_CONFIGURATION_VALUES, "DEFINITION-REF")
  DEFINITION_REF.text = "/AUTOSAR/EcucDefs/CanNm"
  DEFINITION_REF.set("DEST", "ECUC-MODULE-DEF")

  IMPLEMENTATION_CONFIG_VARIANT = ET.SubElement(ECUC_MODULE_CONFIGURATION_VALUES, "IMPLEMENTATION-CONFIG-VARIANT")
  IMPLEMENTATION_CONFIG_VARIANT.text = "VARIANT-POST-BUILD"

  MODULE_DESCRIPTION_REF = ET.SubElement(ECUC_MODULE_CONFIGURATION_VALUES, "MODULE_DESCRIPTION_REF")
  MODULE_DESCRIPTION_REF.text = "/AUTOSAR/EcucDefs/CanNm"
  MODULE_DESCRIPTION_REF.set("DEST", "BSW-IMPLEMENTATION")

  et_elements.append(ET.Element("CONTAINERS"))
  ECUC_MODULE_CONFIGURATION_VALUES.append(et_elements[0])

  return AUTOSAR


In [3]:
def final_out(file_name, et_ele):
  #et_ele = AUTOSAR
  tree = ET.ElementTree(et_ele)

  short_tab = "\t".expandtabs(2)
  ET.indent(tree, space=short_tab, level=0)
  with open (file_name, "wb") as files:
    tree.write(files)

## Implementing the main classes of the generator

- The tag class:

- The parameter class:

- The container classes:

In [4]:
class tag:
    name = ""
    text = ""
    is_sub = -1  # 0 -> element, 1 -> subelement
    in_tag_value = ["", ""]
    et_element_idx = -1  # the index of the et_element in the global array
    par = -1  # the parent index to be used in the case of subelement

    def __init__(self):
      name = ""

    def declare(self): # mostly if definition-ref or value
        # Assumes all attributes are set
        if self.is_sub == 0:
            et_elements.append(ET.Element(self.name))
            self.et_element_idx = len(et_elements) - 1

            if self.text != "":
                et_elements[self.et_element_idx].text = self.text

            if self.in_tag_value[0] != "":
                et_elements[self.et_element_idx].set(self.in_tag_value[0], self.in_tag_value[1])

        else:
            et_subelements.append(ET.SubElement(et_elements[self.par], self.name))
            self.et_element_idx = len(et_subelements) - 1

            if self.text != "":
                et_subelements[self.et_element_idx].text = self.text

            if self.in_tag_value[0] != "":
                et_subelements[self.et_element_idx].set(self.in_tag_value[0], self.in_tag_value[1])

    def get_name(self):
        return self.name

    def set_name(self, value):
        self.name = value

    def get_text(self):
        return self.text

    def set_text(self, value):
        self.text = value

    def get_is_sub(self):
        return self.is_sub

    def set_is_sub(self, value):
        self.is_sub = value

    def get_in_tag_value(self):
        return self.in_tag_value

    def set_in_tag_value(self, value):
        self.in_tag_value = value

    def get_et_element_idx(self):
        return self.et_element_idx

    def set_et_element_idx(self, value):
        self.et_element_idx = value

    def get_par(self):
        return self.par

    def set_par(self, value):
        self.par = value

How to use the parameter class:
- Set def_name, def_ref, value
- Call the declare_parameter function

In [5]:
class parameter(tag):
    def_name = "" # the actual parameter name
    def_ref = tag()
    value = tag()
    param_type = ""  # float or boolean string
    par_cont = -1 # index of parent in the array of containers

    def declare_parameter(self):  # adds the parameter to the file
        # assumes par_cont, param_type, def_name, value.text and self.name is set
        self.is_sub = 0
        super().declare()

        self.def_ref.set_name("DEFINTION-REF")
        self.def_ref.set_par(self.et_element_idx)
        def_ref_in_tag_value = "ECUC-" + self.param_type + "-PARAM-DEF"
        self.def_ref.set_in_tag_value(["DEST", def_ref_in_tag_value])

        # now do the dfs that makes the string of definition ref
        temp_text = "/AUTOSAR/EcucDefs/CanNm/"
        # cur = self.par_cont
        # ancestors = []
        # while cur != -1:
        #     ancestors.append(cur)
        #     cur = containers[cur].par_cont

        # ancestors.reverse()
        # for x in ancestors:
        #     temp_text += "/" + containers[x].short_name.text
        temp_text += self.def_name

        self.def_ref.set_text(temp_text)
        self.def_ref.set_is_sub(1)
        self.def_ref.declare()

        self.value.set_name("VALUE")
        self.value.set_is_sub(1)
        self.value.set_par(self.et_element_idx)
        self.value.declare()

    def get_def_name(self):
        return self.def_name

    def set_def_name(self, value):
        self.def_name = value

    def get_param_value(self):
        return self.param_value

    def set_param_value(self, value):
        self.param_value = value

    def get_par_cont(self):
        return self.par_cont

    def set_par_cont(self, value):
        self.par_cont = value

How to use:
- Instantiate an object
- Set the name, text, in_tag_value
- Set the short_name and def_ref using the setters

In [10]:
class container(tag):
  short_name = tag()
  def_ref = tag()
  #children = [] # array of integers of containers
  container_idx = -1
  par_cont = -1 # parent as a container
  sub_cont_idx = -1
  param_val_idx = -1
  has_param_val = 0
  has_sub_cont = 0

  def declare_container(self):
    # assumes short_name.text is set
    # assumes children and parameters are added
    # (where parameters have their dependencies set except for par_cont and not declared)
    # (and children have their dependencies set except for par_cont and not declared)
    # assumes in_tag_value are given (the set value is UUID)
    # assumes par_cont and container_idx are given
    self.is_sub = 0
    self.name = "ECUC-CONTAINER-VALUE"
    self.declare()

    # create the short_name
    self.short_name.set_name("SHORT-NAME")
    self.short_name.set_is_sub(1)
    self.short_name.set_par(self.et_element_idx)
    self.short_name.declare()

    # create the def-ref
    self.def_ref.set_name("DEFINITION-REF")
    self.def_ref.set_in_tag_value(["DEST", "ECUC-PARAM-CONF-CONTAINER-DEF"])
    self.def_ref.set_is_sub(1)
    self.def_ref.set_par(self.et_element_idx)

    temp_text = "/AUTOSAR/EcucDefs/CanNm/"
    # cur = self.par_cont
    # ancestors = []
    # while cur != -1:
    #     ancestors.append(cur)
    #     cur = containers[cur].par_cont

    # ancestors.reverse()
    # for x in ancestors:
    #     temp_text += containers[x].short_name.text
    temp_text += self.short_name.text

    self.def_ref.set_text(temp_text)
    self.def_ref.declare()

  def add_parameter(self, param):
    if self.has_param_val == 0:
      self.has_param_val = 1
      self.param_val_idx = len(et_elements)
      et_elements.append(ET.Element("PARAMETER-VALUES"))
      et_elements[self.et_element_idx].append(et_elements[self.param_val_idx])
    et_elements[self.param_val_idx].append(et_elements[param.et_element_idx])

  def add_child(self, child):
    if self.has_sub_cont == 0:
      self.has_sub_cont = 1
      self.sub_cont_idx = len(et_elements)
      et_elements.append(ET.Element("SUB-CONTAINERS"))
      et_elements[self.et_element_idx].append(et_elements[self.sub_cont_idx])
    et_elements[self.sub_cont_idx].append(et_elements[child.et_element_idx])

  def get_par_cont(self):
    return self.par_cont

  def set_par_cont(self, value):
    self.par_cont = value

  def get_container_idx(self):
    return self.container_idx

  def set_container_idx(self, value):
    self.container_idx = value

## Unit testing of every class individually

Testing the parameter class

In [8]:
# Reseting the global variables
et_elements = []
et_subelements = []
AS = initialize(et_elements)

# Adding the parameter
x = parameter()
x.def_name = "CanNmMainFunctionPeriod"
x.name = "ECUC-NUMERICAL-PARAM-VALUE"
x.param_type = "FLOAT"
x.value.set_text("False")
x.declare_parameter()

y = parameter()
y.def_name = "CanNmBusLoadReductionEnabled"
y.name = "ECUC-NUMERICAL-PARAM-VALUE"
y.param_type = "BOOLEAN"
y.value.set_text("False")
y.declare_parameter()

# Appending the containers to the prefix of the file
et_elements[0].append(et_elements[x.et_element_idx])
et_elements[0].append(et_elements[y.et_element_idx])

final_out("parameter_testing2.xml", AS)

Testing the container class

In [13]:
# Reseting the global variables
et_elements = []
et_subelements = []
AS = initialize(et_elements)

# Adding the first container with its parameters
c = container()
c.short_name.set_text("CanNmGlobalConfig")
c.set_in_tag_value(["UUID", "e40d0e8a-816a-426a-9159-e229afbced6d"])
c.declare_container()
x = parameter()
x.def_name = "CanNmMainFunctionPeriod"
x.name = "ECUC-NUMERICAL-PARAM-VALUE"
x.param_type = "FLOAT"
x.value.set_text("False")
x.declare_parameter()
c.add_parameter(x)

y = parameter()
y.def_name = "CanNmBusLoadReductionEnabled"
y.name = "ECUC-NUMERICAL-PARAM-VALUE"
y.param_type = "BOOLEAN"
y.value.set_text("False")
y.declare_parameter()
c.add_parameter(y)

# Adding the subcontainer with its parameters
sub_c = container()
sub_c.short_name.set_text("CanNmChannelConfig")
sub_c.set_in_tag_value(["UUID", "2cdaf9a9-575f-4cb1-9a76-d0b312a26cb7"])
sub_c.declare_container()
xx = parameter()
xx.def_name = "CanNmImmediateNmTransmissions"
xx.name = "ECUC-NUMERICAL-PARAM-VALUE"
xx.param_type = "FLOAT"
xx.value.set_text("0")
xx.declare_parameter()
sub_c.add_parameter(xx)
c.add_child(sub_c)

sub_sub_c = container()
sub_sub_c.short_name.set_text("CanNmRxPdu")
sub_sub_c.set_in_tag_value(["UUID", "5d588fa0-8eda-47a2-beb6-796a55542c55"])
sub_sub_c.declare_container()
yy = parameter()
yy.def_name = "CanNmImmediateNmTransmissions"
yy.name = "ECUC-NUMERICAL-PARAM-VALUE"
yy.param_type = "FLOAT"
yy.value.set_text("0")
yy.declare_parameter()
sub_c.add_parameter(yy)
sub_c.add_child(sub_sub_c)

# Appending the containers to the prefix of the file
et_elements[0].append(et_elements[c.et_element_idx])

final_out("containers_testing2.xml", AS)

## Required Data Structures:

- Adjacency list for the containers dependency
- Map for indexing the containers names
- Map for saving the paramenters of each container

In [ ]:
class parameter_item:
  def_name = ""
  value = -1
  data_type = ""
  decl_name = ""
  is_default = 1
  start_range = 1
  end_range = 1
  lower_mult = -1
  upper_mult = -1

  def __init__(self, def_name, value, data_type, decl_name, is_default, start_range, end_range, lower_mult, upper_mult):
    self.def_name = def_name
    self.value = value
    self.data_type = data_type
    self.decl_name = decl_name
    self.is_default = is_default
    self.start_range = start_range
    self.end_range = end_range
    self.lower_mult = lower_mult
    self.upper_mult = upper_mult

In [ ]:
class container_item:
  name = ""
  UUID = ""
  lower_mult = -1
  upper_mult = -1

  def __init__(self, name, UUID, lower_mult, upper_mult):
    self.name = name
    self.UUID = UUID
    self.lower_mult = lower_mult
    self.upper_mult = upper_mult

In [ ]:
container_def = []
par = [-1] * 100000
container_parameters = [[]]

## Parsing the BSWMD

In [ ]:
file_path = 'CanNM_BSWMD.arxml'

tree = etree.parse(file_path)
root = tree.getroot()

In [ ]:
def process_container(ecuc_container):
  name = ecuc_container.find('./SHORT-NAME').text
  LM = ecuc_container.find('./LOWER-MULTIPLICITY').text
  UM = ecuc_container.find('./UPPER-MULTIPLICITY-INFINITE')
  if UM is not None:
    UM = 'inf'
  else:
    UM = ecuc_container.find('./UPPER-MULTIPLICITY').text
  UUID = ecuc_container.get('UUID')
  return name, UUID, LM, UM

In [ ]:
def dfs (ecuc_container, level, p):
  for ecuc_container in ecuc_container.findall('./ECUC-PARAM-CONF-CONTAINER-DEF'):
    name, UUID, LM, UM = process_container(ecuc_container)
    c = container_item(name, UUID, LM, UM)
    container_def.append(c)
    idx = len(container_def) - 1
    par[idx] = p
    for sub_cont in ecuc_container.findall('SUB-CONTAINERS'):
      dfs(sub_cont, level + 1, idx)

In [ ]:
print(root.findall('./CONTAINERS')[0].findall('./ECUC-PARAM-CONF-CONTAINER-DEF'))

[<Element ECUC-PARAM-CONF-CONTAINER-DEF at 0x7cb4a0dde340>]


In [ ]:
containers = root.findall('./CONTAINERS')[0]
dfs(containers, 0, -1)

In [ ]:
print(len(container_def))
for elem in container_def:
  print(elem.name)

7
CanNmGlobalConfig
CanNmChannelConfig
CanNmRxPdu
CanNmTxPdu
CanNmUserDataTxPdu
CanNmPnInfo
CanNmPnFilterMaskByte


## Generating the Dataset

## The Main function

In [ ]:
def print(container_num, mult):
  xxx = initialize()

  index1 = len(containers)
  containers.append(container())
  containers[index1].short_name.set_text(container_def[0].name)
  containers[index1].set_in_tag_value(["UUID", "a816306b-efbe-48b9-ab7f-bda718e857f3"])
  for par in container_parameters[0]:
    p = parameter()
    p.set_def_name(par.def_name)
    p.value.set_text(par.value)
    p.set_param_value(par.data_type)
    p.set_name("ECUC-" + par.decl_name + "-PARAM-VALUE")

  index2 = len(containers)
  containers.append(container())
  containers[index2].short_name.set_text(container_def[1].name)
  containers[index2].set_in_tag_value(["UUID", "a816306b-efbe-48b9-ab7f-bda718e857f3"])
  # for par in container_parameters[1]:
  #     p = parameter()
  #     p.set_def_name(par.def_name)
  #     p.value.set_text(par.value)
  #     p.set_param_value(par.data_type)
  #     p.set_name("ECUC-" + par.decl_name + "-PARAM-VALUE")

  index_list = []
  for i in range(mult):
    index_list.append(len(containers))
    containers.append(container())
    containers[index_list[i]].short_name.set_text(container_def[2].name)
    containers[index_list[i]].set_in_tag_value(["UUID", "a816306b-efbe-48b9-ab7f-bda718e857f3"])
    containers[index_list[i]].declare()
    et_elements[containers[index2].et_element_idx].append(et_elements[containers[index_list[i]].et_element_idx])

  containers[index2].declare()
  et_elements[containers[index1].et_element_idx].append(et_elements[containers[index2].et_element_idx])

  containers[index1].declare()
  et_elements[0].append(et_elements[containers[index1].et_element_idx])

  final_out("sample" + str(mult) + ".xml", xxx)

In [ ]:
def main():
  #for i in range(1, 7):
  print(0, 4)

if __name__ == "__main__":
    main()
